In [1]:
import pandas as pd
from torch.utils.data import DataLoader
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.metrics import balanced_accuracy_score
import matplotlib.pyplot as plt

C:\Users\DELL\.conda\envs\tensorflow\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
# data loading
data = pd.read_csv(r'''D:\A309-21-\课题\DILI Predict\DILI数据+模型\汇总-DILI\train\Padel-Estate Fingerprinterdata.csv''', low_memory=False)
X = data.iloc[:,3:].values
y = data.loc[:,'DILIst.1'].values
#data.head()
# fixed random seed so that training is the same for each division, shuffle: reorder the data before splitting
data_loader = DataLoader(dataset=data, batch_size = 8, shuffle=True, num_workers= 0, drop_last=False)
dataset = data.sample(frac=1).reset_index(drop=True) 
# slicing the dataset in two
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=78, shuffle=True)

In [24]:
KNN = KNeighborsClassifier(n_neighbors =22, weights = 'distance', algorithm='auto',leaf_size = 30, p = 2, metric = 'minkowski')
# cross-validation
KNN_s = cross_val_score(KNN, X_train, y_train, cv = 5, scoring = "precision_weighted").mean()
print("KNN:{}".format(KNN_s))
KNN.fit(X_train, y_train)
# evaluation on test set
y_score_test = KNN.predict(X_test).tolist()  
y_pred_test = (KNN.predict(X_test) > 0.5).astype("int32")     
TN, FP, FN, TP = confusion_matrix(y_test, y_pred_test).ravel()   
SE = TP / (TP + FN)
SP = TN / (TN + FP)
ACC = (TP + TN) / (TP + TN + FP + FN)                    
F1 = (TP / (2 * TP + FN + FP)) * 2
MCC = (TP * TN - FP * FN) / ((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN)) ** 0.5
AUC = roc_auc_score(y_test, y_score_test)
BA = balanced_accuracy_score(y_test, y_pred_test)
LR = pd.DataFrame({'SE': [SE], 'SP': [SP], 'ACC': [ACC], 'AUC': [AUC], 'F1': [F1], 'MCC': [MCC], 'BA': [BA]})
print(LR)

Random Forest:0.6496079318042347
         SE        SP       ACC       AUC        F1       MCC        BA
0  0.862559  0.355769  0.695238  0.609164  0.791304  0.252288  0.609164
